In [274]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from math import log, sqrt, pi, exp

#from pandas_datareader import data
from statsmodels.tsa import stattools
from statsmodels.tsa import arima_model
from statsmodels.graphics import gofplots
from statsmodels.regression import linear_model

from scipy import stats
import scipy.integrate
import scipy.special
# from arch.unitroot import PhillipsPerron
import yfinance as yf
from sklearn.linear_model import LinearRegression

import os
import matplotlib.pyplot as plt  # for plotting
from statsmodels.regression import linear_model

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)
import statistics
import scipy.stats as si
from scipy.stats import norm

Data Download

In [275]:
tickers = ["CSCO","BAC","GE"]
#downoad the prices and volumes for the previously read list of tickers
start_date = '2021-09-01'
end_date = '2022-06-10'
stock_data = yf.download(tickers, start=start_date, end=end_date)
print(stock_data)

[*********************100%***********************]  3 of 3 completed
            Adj Close                             Close             \
                  BAC       CSCO          GE        BAC       CSCO   
Date                                                                 
2021-08-31  40.901463  57.884144  105.141510  41.750000  59.020000   
2021-09-01  40.352840  57.903763  103.395973  41.189999  59.040001   
2021-09-02  40.392227  58.384327  105.989342  41.020000  59.529999   
2021-09-03  40.421772  58.276443  104.483192  41.049999  59.419998   
2021-09-07  40.687637  57.746838  103.026909  41.320000  58.880001   
...               ...        ...         ...        ...        ...   
2022-06-03  36.189999  45.250000   76.970001  36.189999  45.250000   
2022-06-06  36.150002  45.330002   77.000000  36.150002  45.330002   
2022-06-07  36.349998  45.570000   78.000000  36.349998  45.570000   
2022-06-08  35.889999  44.919998   77.160004  35.889999  44.919998   
2022-06-09  34.509998

In [276]:
# Dividend Rates
csco = yf.Ticker("csco")
a_csco = csco.info['dividendRate']
a_csco = a_csco/100
print(a_csco)

ge = yf.Ticker("ge")
a_ge = ge.info['dividendRate']
a_ge = a_ge/100
print(a_ge)

bac = yf.Ticker("bac")
a_bac = bac.info['dividendRate']
a_bac = a_bac/100
print(a_bac)


0.0152
0.0032
0.0084


In [277]:
# Working on statistics
price = stock_data['Adj Close'].values
days = price.shape[0]
ret = price[1:]/price[:-1] - 1
#print(ret)

price_tab = np.array([])
ret_tab = np.array([])
for i in range(len(price)):
    price_tab = np.append(price_tab, price[i])
for i in range(len(ret)):
    ret_tab = np.append(ret_tab, ret[i])

price_tab = price_tab.reshape(days,3)
ret_tab = ret_tab.reshape(days-1,3)

df_price = pd.DataFrame(price_tab)
df_price.columns =['CSCO', 'BAC','GE']
df_ret = pd.DataFrame(ret_tab)
df_ret.columns=['CSCO', 'BAC','GE']
# Print prices
print(df_price)
#Print returns
print(df_ret)

hist_std_csco = np.std(df_ret["CSCO"])*np.sqrt(250)
hist_std_bac = np.std(df_ret["BAC"])*np.sqrt(250)
hist_std_ge = np.std(df_ret["GE"])*np.sqrt(250)

# Historical volatility
print(hist_std_csco)
print(hist_std_bac)
print(hist_std_ge)

          CSCO        BAC          GE
0    40.901463  57.884144  105.141510
1    40.352840  57.903763  103.395973
2    40.392227  58.384327  105.989342
3    40.421772  58.276443  104.483192
4    40.687637  57.746838  103.026909
..         ...        ...         ...
191  36.189999  45.250000   76.970001
192  36.150002  45.330002   77.000000
193  36.349998  45.570000   78.000000
194  35.889999  44.919998   77.160004
195  34.509998  44.259998   74.779999

[196 rows x 3 columns]
         CSCO       BAC        GE
0   -0.013413  0.000339 -0.016602
1    0.000976  0.008299  0.025082
2    0.000731 -0.001848 -0.014210
3    0.006577 -0.009088 -0.013938
4   -0.013069 -0.003397 -0.006874
..        ...       ...       ...
190 -0.013897 -0.011793 -0.013205
191 -0.001105  0.001768  0.000390
192  0.005532  0.005294  0.012987
193 -0.012655 -0.014264 -0.010769
194 -0.038451 -0.014693 -0.030845

[195 rows x 3 columns]
0.3023973061156119
0.28847258631780737
0.3305850811985709


In [278]:
# Getting Call/Put options: Market Prices
opt_csco = csco.option_chain(date='2023-01-20')
options_cisco = opt_csco
print(options_cisco)

opt_bac = bac.option_chain(date='2023-01-20')
options_bac = opt_bac

opt_ge = ge.option_chain(date='2023-01-20')
options_ge = opt_ge

Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice    bid  \
0   CSCO230120C00020000 2022-06-08 16:28:16+00:00    20.0      25.48  23.15   
1   CSCO230120C00023000 2022-01-03 18:48:29+00:00    23.0      39.95  33.05   
2   CSCO230120C00025000 2022-05-19 17:09:20+00:00    25.0      16.57  18.25   
3   CSCO230120C00028000 2022-06-10 19:33:20+00:00    28.0      16.00  15.45   
4   CSCO230120C00030000 2022-05-24 19:19:18+00:00    30.0      14.20  13.60   
5   CSCO230120C00033000 2022-06-10 17:55:00+00:00    33.0      11.50  11.10   
6   CSCO230120C00035000 2022-06-10 18:53:52+00:00    35.0       9.86   9.45   
7   CSCO230120C00037000 2022-06-08 14:12:30+00:00    37.0       9.47   7.95   
8   CSCO230120C00040000 2022-06-10 18:27:11+00:00    40.0       6.15   5.80   
9   CSCO230120C00042000 2022-06-10 19:46:24+00:00    42.0       4.75   4.60   
10  CSCO230120C00045000 2022-06-13 15:32:41+00:00    45.0       3.15   3.15   
11  CSCO230120C00047000 2022-06-13 15:

In [279]:
# Get Market Call Prices - CSCO
strike_call_list_cisco = (options_cisco[0]['strike'])
market_price_call_cisco = (options_cisco[0]['lastPrice'])
market_implied_vol_call_cisco = (options_cisco[0]['impliedVolatility'])
df_market_call_csco = pd.DataFrame({'strike':strike_call_list_cisco ,
                   'lastPrice':market_price_call_cisco ,
                   'impliedVolatility': market_implied_vol_call_cisco})
#print(df_market_call)

# Get Market Put Prices - CSCO
strike_put_list_cisco = (options_cisco[1]['strike'])
market_price_put_cisco = (options_cisco[1]['lastPrice'])
market_implied_vol_put_cisco = (options_cisco[1]['impliedVolatility'])
df_market_put_cisco = pd.DataFrame({'strike':strike_put_list_cisco ,
                   'lastPrice':market_price_put_cisco ,
                   'impliedVolatility': market_implied_vol_put_cisco})

# Get Market Call Prices - GE
strike_call_list_ge = (options_ge[0]['strike'])
market_price_call_ge = (options_ge[0]['lastPrice'])
market_implied_vol_call_ge = (options_ge[0]['impliedVolatility'])
df_market_call_ge = pd.DataFrame({'strike':strike_call_list_ge ,
                   'lastPrice':market_price_call_ge ,
                   'impliedVolatility': market_implied_vol_call_ge})
#print(df_market_call)

# Get Market Put Prices - GE
strike_put_list_ge = (options_ge[1]['strike'])
market_price_put_ge = (options_ge[1]['lastPrice'])
market_implied_vol_put_ge = (options_ge[1]['impliedVolatility'])
df_market_put_ge = pd.DataFrame({'strike':strike_put_list_ge ,
                   'lastPrice':market_price_put_ge ,
                   'impliedVolatility': market_implied_vol_put_ge})

#print(df_market_call_ge)

Parameters: Black-Scholes

In [280]:
S_0_CSCO = 43.31            # stock price: CSCO
S_0_GE = 67.88              # stock price: GE
r = 1.40/100                # interest rate

sigma_CSCO = hist_std_csco  # historical (std) volatility CSCO
sigma_GE = hist_std_ge      # historical (std) volatility GE

K_CSCO = 55                 # specific strike CSCO
K_GE = 10                   # specific strike GE
T = 150/250                 # 30 weeks: 150 trading days
N = 10000                   # number of trajectories

# Getting the quoted implied volatility: CSCO
market_call_CSCO = df_market_call_csco.loc[df_market_call_csco['strike']==K_CSCO]
implied_vol_CSCO_Call = (np.array(market_call_CSCO["impliedVolatility"]))
implied_vol_CSCO_Call = implied_vol_CSCO_Call[0]

market_put_CSCO = df_market_put_cisco.loc[df_market_put_cisco['strike']==K_CSCO]
implied_vol_CSCO_Put = (np.array(market_put_CSCO["impliedVolatility"]))
implied_vol_CSCO_Put = implied_vol_CSCO_Put[0]

# Getting the quoted implied volatility: GE
market_call_ge = df_market_call_ge.loc[df_market_call_ge['strike']==K_GE]
implied_vol_ge_Call = (np.array(market_call_ge["impliedVolatility"]))
implied_vol_ge_Call = implied_vol_ge_Call[0]

market_put_ge = df_market_put_ge.loc[df_market_put_ge['strike']==K_GE]
implied_vol_ge_Put = (np.array(market_put_ge["impliedVolatility"]))
implied_vol_ge_Put = implied_vol_ge_Put[0]

Black-Scholes-Merton: Options Prices

In [281]:
def black_scholes_merton_call_div(S, K, T, r, a, sigma):

    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #a: dividend yield
    #sigma: volatility of underlying asset

    d1 = (np.log(S/K) + (r-a+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = (np.log(S/K) + (r-a-0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    call = (S*np.exp(-a*T) * si.norm.cdf(d1, 0.0, 1.0) - K*np.exp(-r*T) * si.norm.cdf(d2, 0.0, 1.0))
    return call


def black_scholes_put_div(S, K, T, r, a, sigma):

    d1_bs = (np.log(S/K) + (r-a+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2_bs = (np.log(S/K) + (r-a-0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    put = (K*np.exp(-r*T) * si.norm.cdf(-d2_bs, 0.0, 1.0) - S*np.exp(-a*T) * si.norm.cdf(-d1_bs, 0.0, 1.0))
    return put

In [282]:
# Black-Scholes CSCO results
call_bs_CSCO =  black_scholes_merton_call_div(S_0_CSCO, K_CSCO, T, r, a_csco, hist_std_csco)
put_bs_CSCO =  black_scholes_put_div(S_0_CSCO, K_CSCO, T, r, a_csco, hist_std_csco)

call_bs_CSCO_implied_vol =  black_scholes_merton_call_div(S_0_CSCO, K_CSCO, T, r, a_csco, implied_vol_CSCO_Call)
put_bs_CSCO_implied_vol =  black_scholes_put_div(S_0_CSCO, K_CSCO, T, r, a_csco, implied_vol_CSCO_Put)

# Black-Scholes GE results
call_bs_GE =  black_scholes_merton_call_div(S_0_GE, K_GE, T, r, a_ge, hist_std_ge)
put_bs_GE =  black_scholes_put_div(S_0_GE, K_GE, T, r, a_ge, hist_std_ge)

call_bs_GE_implied_vol =  black_scholes_merton_call_div(S_0_GE, K_GE, T, r, a_ge, implied_vol_ge_Call)
put_bs_GE_implied_vol =  black_scholes_put_div(S_0_GE, K_GE, T, r, a_ge, implied_vol_ge_Put)

Black-Scholes-Merton: Greeks


In [283]:
def d1(S,K,T,r,a,sigma):
    return(log(S/K)+(r-a+sigma**2/2.)*T)/(sigma*sqrt(T))

def d2(S,K,T,r,a,sigma):
    return d1(S,K,T,r,a,sigma)-sigma*sqrt(T)

############## CALL OPTION ##############

def call_delta(S,K,T,r,a,sigma):
    return exp(-a*T)*norm.cdf(d1(S,K,T,r,sigma))

def call_gamma(S,K,T,r,a,sigma):
    return exp(-a*T)*norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))

def call_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))

############## PUT OPTION ##############

def put_delta(S,K,T,r,a,sigma):
    return -exp(-a*T)*norm.cdf(-d1(S,K,T,r,sigma))

def put_gamma(S,K,T,r,a,sigma):
    return exp(-a*T)*norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))

def put_vega(S,K,T,r,a,sigma):
    return 0.01*exp(-a*T)*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))

Monte-Carlo Simulation


In [284]:
# # Monte-Carlo Simulation:
#
# # Parameters
# T = 1.0
# N = 10000
# S_0 = 50
# sigma = 0.20
#
# # Example one-period simulation
#
# # lognormal distribution of the price
#
# Z = np.random.normal(size=N)
# S_T = S_0*np.exp((r-0.5*sigma**2)*T+sigma*np.sqrt(T)*Z)
# plt.hist(S_T, bins='auto')
# plt.xlabel('price')
# plt.ylabel('frequency')

In [285]:
# Monte Carlo simulation: European Option

call_payoff_maturity_CSCO = []
put_payoff_maturity_CSCO = []
call_payoff_maturity_GE = []
put_payoff_maturity_GE = []


days_test = 150 # 1 week = 5 trading days
dt = 1/250

S_CSCO = np.zeros((days_test+1, N))
S_CSCO[0] = S_0_CSCO
S_GE = np.zeros((days_test+1, N))
S_GE[0] = S_0_GE

for t in range(1, days_test+1):
    Z_CSCO = np.random.normal(size=N)
    Z_GE = np.random.normal(size=N)
    S_CSCO[t] = S_CSCO[t-1]*np.exp((r-a_csco-0.5*sigma_CSCO**2)*dt+sigma_CSCO*np.sqrt(dt)*Z_CSCO)
    S_GE[t] = S_GE[t-1]*np.exp((r-a_ge-0.5*sigma_GE**2)*dt+sigma_GE*np.sqrt(dt)*Z_GE)

Valuation (European)

In [286]:
S_T_CSCO = S_CSCO[-1]
S_T_GE = S_GE[-1]
for i in range(len(S_T_CSCO)):
    call_payoff_maturity_CSCO.append(max(S_T_CSCO[i]-K_CSCO,0))
    put_payoff_maturity_CSCO.append(max(K_CSCO-S_T_CSCO[i],0))

    call_payoff_maturity_GE.append(max(S_T_GE[i]-K_GE,0))
    put_payoff_maturity_GE.append(max(K_GE-S_T_GE[i],0))

# Price of CSCO options: expiry on Jan 20, 2023
VMC_call_CSCO = np.mean(call_payoff_maturity_CSCO)
C_0_CSCO = VMC_call_CSCO*np.exp(-r*days_test/250)
print(C_0_CSCO)

VMC_put_CSCO = np.mean(put_payoff_maturity_CSCO)
P_0_CSCO = VMC_put_CSCO*np.exp(-r*days_test/250)
print(P_0_CSCO)

# Price of GE options: expiry on Jan 20, 2023
VMC_call_GE = np.mean(call_payoff_maturity_GE)
C_0_GE = VMC_call_GE*np.exp(-r*days_test/250)
print(C_0_GE)

VMC_put_GE = np.mean(put_payoff_maturity_GE)
P_0_GE = VMC_put_GE*np.exp(-r*days_test/250)
print(P_0_GE)

0.8994707322903152
12.467650564747087
58.14714359809465
0.0


Recap Results: Market Prices, Black-Scholes formulas, Black-Scholes Monte-Carlo

In [287]:
# Market Prices
market_call_CSCO = df_market_call_csco.loc[df_market_call_csco['strike']==K_CSCO]
market_put_CSCO = df_market_put_cisco.loc[df_market_put_cisco['strike']==K_CSCO]

print("CSCO Call prices:")
print("CSCO Market call: ", market_call_CSCO)
print("CSCO Black-Scholes call with implied volatility: ", call_bs_CSCO_implied_vol)
print("CSCO Black-Scholes call with historical volatility: ", call_bs_CSCO)
print("EU CSCO call with MC estimation: ", C_0_CSCO)
# print()
# print("CSCO Put prices:")
# print("CSCO Market put: ", market_put_CSCO)
# print("CSCO Black-Scholes put with implied volatility: ", put_bs_CSCO_implied_vol)
# print("CSCO Black-Scholes put with historical volatility: ", put_bs_CSCO)
# print("EU CSCO put with MC estimation: ", P_0_CSCO)


CSCO Call prices:
CSCO Market call:      strike  lastPrice  impliedVolatility
14    55.0       0.62           0.270759
CSCO Black-Scholes call with implied volatility:  0.6370730683225441
CSCO Black-Scholes call with historical volatility:  0.8992421320728328
EU CSCO call with MC estimation:  0.8994707322903152


In [288]:
market_call_ge = df_market_call_ge.loc[df_market_call_ge['strike']==K_GE]
market_put_ge = df_market_put_ge.loc[df_market_put_ge['strike']==K_GE]

# print("GE Call prices:")
# print("GE Market call: ", market_call_ge)
# print("GE Black-Scholes call with implied volatility: ", call_bs_GE_implied_vol)
# print("GE Black-Scholes call with historical volatility: ", call_bs_GE)
# print("EU GE call with MC estimation: ", C_0_GE)
# print()
print("GE Put prices:")
print("GE Market put: ", market_put_ge)
print("GE Black-Scholes call with implied volatility: ", put_bs_GE_implied_vol)
print("GE Black-Scholes put with historical volatility: ", put_bs_GE)
print("EU GE put with MC estimation: ", P_0_GE)

GE Put prices:
GE Market put:     strike  lastPrice  impliedVolatility
5    10.0       0.87           1.654299
GE Black-Scholes call with implied volatility:  0.8404959214012679
GE Black-Scholes put with historical volatility:  2.620124493956341e-14
EU GE put with MC estimation:  0.0
